In [111]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

#from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE 

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

np.random.seed(42)

import random
random.seed(42)

### Spliting data into known and unknown labels

In [94]:
data = pd.read_csv('./data/raw_data/data.csv', header=None)
labels = pd.read_csv('./data/raw_data/labels.csv', header=None)

data_labelled = data[:len(labels.index)]
data_unlabelled = data[len(labels.index):]

data_labelled.to_csv('./data/processed_data/known_labels.csv')
data_unlabelled.to_csv('./data/processed_data/unknown_labels.csv')

### Merge known labels dataframe with the respective labels

In [95]:
labels = labels.rename(columns={0: 'y'})
df = pd.concat([data_labelled, labels], axis=1)

df['y'] = df['y'].map({1: False, 2: True})

df.to_csv('./data/processed_data/data_with_labels.csv')

### Class ratio and null values

In [96]:
print('Class ratio:\n', labels['y'].value_counts())
print('Columns with null values:\n', df.columns[df.isna().any()].tolist())

Class ratio:
 1    156
2     23
Name: y, dtype: int64
Columns with null values:
 []


It seems like theres a high class imbalance however no columns have null values

### Create a vizualization html page for our dataset

In [5]:
profile = ProfileReport(df, minimal=True)
profile.to_file("visualization/output.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 179.11it/s]


## Decision Trees

From our exploratory analysis it seems like ANOVA and Relief work the best. Lets start with using ANOVA classification to get the top features and apply SMOTE to try and combat the class imbalance.

Build a pipeline with the ANOVA feature selector > SMOTE > Decision Tree Classifier

In [98]:
tree_pipeline = Pipeline(
    [
     ('selector',SelectKBest(f_classif)),
     ('smote',SMOTE()),
     ('model',DecisionTreeClassifier())
    ]
)

### Grid Search

Define the parameters that we'll want to test during our 5 fold cross validation. Lets use F1 score for measuring the performance of our model given the class imbalance.

In [99]:
tree_search = GridSearchCV(
    estimator = tree_pipeline,
    param_grid = {'selector__k':[5,10,15,20,30,40,50],
    'model__criterion':['gini','entropy'],
    'model__max_depth':range(1,20),
    },
    cv=5,
    n_jobs=-1,
    scoring='f1',
    verbose=1
)

Restricting max_depth should help with preventing overfitting along with a 5 fold cross validation

### Results

In [103]:
tree_search.fit(data_labelled,labels.values.ravel())

#y_pred = search.best_estimator_.predict(data_labelled)

print('Best parameters:\n', tree_search.best_params_)
print('Best score f1:\n',tree_search.best_score_)

Fitting 5 folds for each of 266 candidates, totalling 1330 fits
Best parameters:
 {'model__criterion': 'gini', 'model__max_depth': 17, 'selector__k': 50}
Best score f1:
 0.9809475019516004


## K-Nearest Neighbors 

Given that KNN relies on majority voting based on class membership of k nearest neighbors, we should use a feature scaller as the first step on our pipeline.

In [112]:
knn_pipeline = Pipeline(
    [('scaller',StandardScaler()),
     ('selector',SelectKBest(f_classif)),
     ('smote',SMOTE()),
     ('model',KNeighborsClassifier())
    ]
)

In [113]:
knn_search = GridSearchCV(
    estimator = knn_pipeline,
    param_grid = {'selector__k':[5,10,15,20,30,40,50],
    'model__weights':['uniform','distance'],
    'model__n_neighbors':[1,3,5,7,9],
    'model__metric':['euclidean','manhattan','chebyshev','minkowski']
    },
    cv=5,
    n_jobs=-1,
    scoring='f1',
    verbose=1
)

In [116]:
knn_search.fit(data_labelled,labels.values.ravel())

print('Best parameters:\n', knn_search.best_params_)
print('Best score f1:\n',knn_search.best_score_)

Fitting 5 folds for each of 280 candidates, totalling 1400 fits
Best parameters:
 {'model__metric': 'chebyshev', 'model__n_neighbors': 7, 'model__weights': 'distance', 'selector__k': 40}
Best score f1:
 0.9968253968253968
